In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
print(df.shape)
df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df.duplicated(keep=False).sum()

824

In [4]:
df.drop_duplicates(inplace=True)

In [5]:
df.dtypes

review       object
sentiment    object
dtype: object

In [6]:
df.sentiment.value_counts()

positive    24884
negative    24698
Name: sentiment, dtype: int64

In [7]:
df.replace({'positive':1, 'negative':0}, inplace=True)
df.sentiment.value_counts()

1    24884
0    24698
Name: sentiment, dtype: int64

In [8]:
df.dtypes

review       object
sentiment     int64
dtype: object

In [9]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[A-Za-z]+')
df['text_tokenized'] = df.review.map(lambda review: tokenizer.tokenize(review))

In [10]:
df.head()

,review,sentiment,text_tokenized
0,One of the other reviewers has mentioned that ...,1,"[One, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. <br /><br />The...,1,"[A, wonderful, little, production, br, br, The..."
2,I thought this was a wonderful way to spend ti...,1,"[I, thought, this, was, a, wonderful, way, to,..."
3,Basically there's a family where a little boy ...,0,"[Basically, there, s, a, family, where, a, lit..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[Petter, Mattei, s, Love, in, the, Time, of, M..."


In [11]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
df['text_stemmed'] = df['text_tokenized'].map(lambda x: [stemmer.stem(word) for word in x])

In [12]:
df.head()

,review,sentiment,text_tokenized,text_stemmed
0,One of the other reviewers has mentioned that ...,1,"[One, of, the, other, reviewers, has, mentione...","[one, of, the, other, review, has, mention, th..."
1,A wonderful little production. <br /><br />The...,1,"[A, wonderful, little, production, br, br, The...","[a, wonder, littl, product, br, br, the, film,..."
2,I thought this was a wonderful way to spend ti...,1,"[I, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonder, way, to, sp..."
3,Basically there's a family where a little boy ...,0,"[Basically, there, s, a, family, where, a, lit...","[basic, there, s, a, famili, where, a, littl, ..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[Petter, Mattei, s, Love, in, the, Time, of, M...","[petter, mattei, s, love, in, the, time, of, m..."


In [13]:
df['text_sent'] = df['text_stemmed'].map(lambda l: ' '.join(l))

In [14]:
df.head()

,review,sentiment,text_tokenized,text_stemmed,text_sent
0,One of the other reviewers has mentioned that ...,1,"[One, of, the, other, reviewers, has, mentione...","[one, of, the, other, review, has, mention, th...",one of the other review has mention that after...
1,A wonderful little production. <br /><br />The...,1,"[A, wonderful, little, production, br, br, The...","[a, wonder, littl, product, br, br, the, film,...",a wonder littl product br br the film techniqu...
2,I thought this was a wonderful way to spend ti...,1,"[I, thought, this, was, a, wonderful, way, to,...","[i, thought, this, was, a, wonder, way, to, sp...",i thought this was a wonder way to spend time ...
3,Basically there's a family where a little boy ...,0,"[Basically, there, s, a, family, where, a, lit...","[basic, there, s, a, famili, where, a, littl, ...",basic there s a famili where a littl boy jake ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[Petter, Mattei, s, Love, in, the, Time, of, M...","[petter, mattei, s, love, in, the, time, of, m...",petter mattei s love in the time of money is a...


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
features = cv.fit_transform(df.text_sent)

In [16]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(features, df.sentiment, test_size=0.2, random_state=0, shuffle=True)

In [17]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(Xtrain, ytrain)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [18]:
ypred = log_reg.predict(Xtest)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, ypred)

0.8788948270646365